In [40]:
import pickle
import networkx as nx
import numpy as np

In [41]:
with open('GOT-characters-raw.pickle','rb') as fp:
    characters = pickle.load(fp)
len(characters.keys())

340

In [86]:
entities = set()
for chapter in characters.values():
    for entity in chapter:
        name = entity[0]
        entities.add(name)
print('analysis contains',len(entities),'individual entities')

analysis contains 5522 individual entities


Name: 
Type: Graph
Number of nodes: 0
Number of edges: 0



In [79]:
def get_edges_from_chapter(chapter):
    chapter.sort(key=lambda a:a[1],reverse=False)

    potential_edges = []
    for i in range(0,len(chapter)):
        name1,start1,end1 = chapter[i]
        for j in range(i+1,len(chapter)):    
            name2,start2,end2 = chapter[j]
            if name2 != name1:
                potential_edges.append((name1,name2,{'dist':start2-end1}))

    edges = {}
    for name1,name2,value in potential_edges:
        if (name1,name2) not in edges.keys():
            if (name2,name1) not in edges.keys():
                edges[(name1,name2)] = {'values':[value['dist']]}
            else:
                edges[(name2,name1)]['values'].append(value['dist'])
        else:
            edges[(name1,name2)]['values'].append(value['dist'])

    for k,v in edges.items():
        #edges[k]['weight']=np.percentile(v['values'],10)
        # the 448 value is the mean + standard deviation of the length of a paragraph in game of thrones books 
        edges[k]['weight']=len([a for a in v['values'] if a <448])
    
    ready_for_nx = []
    for k,v in edges.items():
        name1 = k[0]
        name2 = k[1]
        weight = v['weight']
        if weight > 0:
            ready_for_nx.append((name1,name2,{'weight':weight}))
    return ready_for_nx

In [82]:
GOT = nx.Graph()

In [83]:
for chapter in characters.values():
    GOT.add_edges_from(get_edges_from_chapter(chapter))

In [84]:
print(nx.info(GOT))

Name: 
Type: Graph
Number of nodes: 5515
Number of edges: 88598
Average degree:  32.1298


In [85]:
nx.write_gexf(GOT,'GOT-version-1.gexf')